In [4]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features, reduce_mem_usage, elo_save_feature
from preprocessing import get_dummies
import datetime

from tqdm import tqdm
import time
import sys
from joblib import Parallel, delayed

key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

In [5]:
df_hist = pd.read_csv('../input/historical_transactions.csv')
df_hist['purchase_amount_new'] = np.round(df_hist['purchase_amount'] / 0.00150265118 + 497.06, 2)
df_hist['installments'] = df_hist['installments'].map(lambda x:  
                                                    1 if x<1 else
                                                    1 if x>100 else
                                                      x
                                                     )

In [6]:
#========================================================================
# Dataset Load 
use_cols = [key, 'authorized_flag', 'installments', 'merchant_category_id', 'merchant_id', 'subsector_id', 'month_lag', 'purchase_amount_new', 'purchase_date']
df_hist = df_hist[use_cols]

auth1 = df_hist[df_hist['authorized_flag']=='Y']

def get_new_columns(name,aggs):
    if len(name):
        return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]
    else:
        return [k + '_' + agg for k in aggs.keys() for agg in aggs[k]]
#========================================================================

In [50]:
debug = [True, False][1]
#========================================================================
# Merchant id 別の集計
# month_lagで集計する（purchase_dateは別にやる）
df = auth1
if debug:
    df = df.head(100000)
    
# Args Setting
level = [key, 'merchant_id', 'month_lag']
#========================================================================

#========================================================================
# Aggregation
print("Aggregation Start!!")

aggs = {}
aggs['purchase_amount_new'] = ['sum']
aggs['installments'] = ['mean', 'max', 'sum']
df_agg = df.groupby(level).agg(aggs)

new_cols = get_new_columns(name='', aggs=aggs)
df_agg.columns = new_cols
df_agg[f'purchase_amount_new_sum_per_installments_sum'] = df_agg[f'purchase_amount_new_sum'] / df_agg[f'installments_sum']

mer_lag_cnt = df.groupby([key, 'merchant_id'])['month_lag'].nunique().reset_index().rename(columns={'month_lag':'month_lag_cnt'})
df_agg = df_agg.reset_index().merge(mer_lag_cnt, how='inner', on=[key, 'merchant_id'])
#========================================================================

#========================================================================
# multi month_lagをもつmerchantに絞る
print("Aggregate Term Setting")
df_merchant = df_agg[df_agg['month_lag_cnt']>1]
del df_agg
gc.collect()
df_merchant.drop(['month_lag_cnt'], axis=1, inplace=True)
#========================================================================
    
#========================================================================
# oldに存在するがnewにいないcard_id, merchantをnewにもたせる。
feat_cols = [col for col in df_merchant.columns if col.count('amount') or col.count('install')]
df_merchant.sort_values(by=[key, 'merchant_id', 'month_lag'], inplace=True)
df_merchant['month_lag_-1'] = df_merchant['month_lag']+1
df_merchant = df_merchant.merge(df_merchant, how='left', left_on=[key, 'merchant_id', 'month_lag'], right_on=[key, 'merchant_id', 'month_lag_-1'])

df_merchant.rename(columns={'month_lag_x':'month_lag'}, inplace=True)
for col in feat_cols:
    df_merchant[f"ratio_lag1_{col}"] = df_merchant[col+'_x'] /  df_merchant[col+'_y']
    df_merchant[f"ratio_lag1_{col}"] = df_merchant[f"ratio_lag1_{col}"].map(lambda x: 10 if x==np.inf or x==-np.inf else x)
xy_cols = [col for col in df_merchant.columns if col.count('_x') or col.count('_y')]

df_merchant.drop(xy_cols, axis=1, inplace=True)
#========================================================================

#========================================================================
# ratio>1のmerchantがいくつあるか
# 直近月と直近3ヶ月の, ratio>1となったmerchantの割合に対して
lag_list = [0, -2]
tmp_list = []
base_cols = []
for lag_num in lag_list:
    lag0 = df_merchant[df_merchant['month_lag']>=lag_num].set_index([key, 'merchant_id']).drop('month_lag', axis=1)
    tmp = ((lag0>1)*1).reset_index().groupby([key, 'merchant_id']).sum()
    tmp = tmp.groupby(key).sum()
    if len(base_cols)==0:
        base_cols = tmp.columns
    tmp.columns = [f"up_cnt_Mlag{lag_num}_{col}" for col in tmp.columns]
    tmp_list.append(tmp)
df_lag = pd.concat(tmp_list, axis=1)

for col in base_cols:
    df_lag[f"up_cnt_ratio_lag0_-2_{col}"] = df_lag[f"up_cnt_Mlag0_{col}"] / df_lag[f"up_cnt_Mlag-2_{col}"]

feat_cols = [col for col in df_lag.columns if col.count('Mlag0') or col.count('up_cnt_ratio')]
#========================================================================

#========================================================================
# Save Feature
prefix = '247_rad'
    
print(f"{prefix} Feature Saving...")

base = utils.read_df_pkl('../input/base_no_out*')
base = base[[key, target]].set_index(key)
base = base.join(df_lag)
base.fillna(-1, inplace=True)
del df_lag
gc.collect()

elo_save_feature(df_feat=base, prefix=prefix)
print('Complete!')
#========================================================================

Aggregation Start!!
Aggregate Term Setting


  0%|          | 0/1 [00:00<?, ?it/s]

247_rad Feature Saving...


100%|██████████| 1/1 [00:00<00:00,  3.91it/s]


Complete!
